In [1]:
import sys
sys.path.append('../../preprocess_assets/')

In [2]:
import glob
import os
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM, Embedding
import tensorflow as tf
from features_extraction import *
from data_shuffling_split import *
from ara_vec_preprocess_configs import *
from ml_modeling import *
from keras_models import *

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
strat_train_set = read_file("train/strat_train_set.csv")
strat_train_set.head()

,text,label,classes
0,راجعت الردود فوجدت المتزمتين دينيا هم الاكثر ا...,0,NEG
1,#شاهد_سكاي : #عيد_العمال تراجع اعدد العاطلين,2,OBJ
2,يقوم د / ابو الفتوح التواصل مع مختلف التيارات ...,2,OBJ
3,هل الشرطه والجيش سيطبق عليهم حدود الاجور دي ؟,2,OBJ
4,1 -انسحاب الوفد المصري المشارك في مهرجان مالم...,2,OBJ


In [4]:
x_train_text, x_val_text, y_train, y_val = prepare_data(strat_train_set)

The number of instances in the training data after StratifiedShuffleSplit are:  9608
The number of instances in the testing data after StratifiedShuffleSplit are:   197
The number of trainin instances:  9608
The number of validation instances:  197
The number of trainin labels :  9608
The number of validation labels :  197


In [5]:
x_train_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(x_train_text)

print("Before Tokenization : \n", x_train_text[:3])
print("="*50)
print("After Tokenization : \n", x_train_text_tokenized[:3])
print("="*50)

x_val_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(x_val_text)

print("Before Tokenization : \n", x_val_text[:3])
print("="*50)
print("After Tokenization : \n", x_val_text_tokenized[:3])

fullgram_x_train_text_tokenized = get_all_ngrams(x_train_text_tokenized)
print("full gram tokenization : \n", fullgram_x_train_text_tokenized[:3])
print("="*50)
fullgram_x_val_text_tokenized = get_all_ngrams(x_val_text_tokenized)
print("full gram tokenization : \n", fullgram_x_val_text_tokenized[:3])
print("="*50)

Before Tokenization : 
 ['#فتاه_تغتصب_شباب_كانوا_في_طريقهم_لمنزلهم يمكن بويه ‹☺›', 'تابعوني علي قناه العربيه التاسعه مساء باذن الله', 'هل سنري اليوم الذي نعود فيه الي حياه طبيعيه دون مظاهرات او اعتصامات او اضراباتو لا اتهامات و محاكمات و انتقام ؟ نرجع الي عملنا و حياتنا']
After Tokenization : 
 [['#', 'فتاه_تغتصب_شباب_كانوا_في_طريقهم_لمنزلهم', 'يمكن', 'بويه', '‹☺›'], ['تابعوني', 'علي', 'قناه', 'العربيه', 'التاسعه', 'مساء', 'باذن', 'الله'], ['هل', 'سنري', 'اليوم', 'الذي', 'نعود', 'فيه', 'الي', 'حياه', 'طبيعيه', 'دون', 'مظاهرات', 'او', 'اعتصامات', 'او', 'اضراباتو', 'لا', 'اتهامات', 'و', 'محاكمات', 'و', 'انتقام', '؟', 'نرجع', 'الي', 'عملنا', 'و', 'حياتنا']]
Before Tokenization : 
 ['قصه شاب مسيحي يشارك معنا في صناع الحياه تحيه كبيره الي سامي والي 100 مسيحي يشاركون معنا في معسكر ابو قير،', '#سري ويرد الصباح علي كلام سعود الفيصل ( انا لن اقبل تهديدك لي وللكويتين بهذه اللهجه ) ', 'ليه يوصلوا الواحد في سنه انه يكره عيشته واليوم اللي اتولد فيه ؟ #ثانويه_عامه #علمونا_صح']
After Tokenization : 


# Our CBOW Word2Vec Model

In [6]:
our_word2vec_model = load_word2vec_model("../word2vec_models/rezk/skipgram_NS/sk_gr_negative_sampeling_fullgram_vec_size_300-d_min_count_100")

In [7]:
number_of_features = 300
max_len_str = 132
word2vec_path = "rezk/cbow/"
model_path_to_save = "../ml_models_saved/"
hid_num_neurons = 25
learning_rate = .00005
epochs = 10
estimators = voting_models()

performance_lr = keras.callbacks.ReduceLROnPlateau(factor=.5, patience=5)
RMSprop_optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate, rho=.9)


X_train_embed_matrix = text_to_matrix_using_word2vec(our_word2vec_model, fullgram_x_train_text_tokenized, max_len_str)
X_val_embed_matrix = text_to_matrix_using_word2vec(our_word2vec_model, fullgram_x_val_text_tokenized, max_len_str)
# Reshape because of deep learning model
X_train_embed_matrix = X_train_embed_matrix.reshape(X_train_embed_matrix.shape[0], max_len_str, number_of_features)
X_val_embed_matrix = X_val_embed_matrix.reshape(X_val_embed_matrix.shape[0], max_len_str, number_of_features)

(9608, 132, 300)
(9608, 39600)
[ 0.005478 -0.04172   0.005714 -0.03497  -0.1553    0.1969    0.05286
  0.2722    0.3384   -0.2032   -0.1765   -0.05478  -0.1627    0.1953
 -0.001688 -0.08417   0.1536   -0.3528   -0.1882   -0.1313    0.2078
 -0.0564   -0.2031    0.02293   0.1746   -0.1052    0.007717  0.05777
 -0.06696  -0.1255    0.01427   0.0869    0.1155    0.1318   -0.0492
 -0.01573  -0.1229    0.07776  -0.0928   -0.1912    0.05603  -0.01492
  0.01863  -0.2324    0.002361  0.2047    0.11084   0.1925   -0.0379
 -0.2247  ]
(197, 132, 300)
(197, 39600)
[ 0.2832   -0.2688   -0.010796  0.2869    0.2957   -0.1874   -0.25
  0.04913   0.2607   -0.32     -0.07385  -0.0223    0.2112   -0.2418
 -0.0984    0.186    -0.01915   0.0668    0.1037   -0.3662   -0.05264
 -0.2766   -0.1348    0.03833  -0.3352    0.2297   -0.05524  -0.1621
 -0.1427    0.1648    0.2405   -0.1508   -0.0077    0.1161   -0.156
  0.1943    0.1304    0.1466    0.285    -0.315    -0.1134    0.06235
  0.264    -0.1782    0.169  

# Our CBOW Word2Vec Model

# With  Rmsprob and  Batch Normalization

In [8]:
callbacks_ = keras_callbacks(word2vec_type="rezk_skipgram_NS_word2vec", model_type="Rmsprob_lstm_with_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)
model = lstm_with_batch_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, RMSprop_optimizer)
model.summary()

rezk_skipgram_NS_word2vec_Rmsprob_lstm_with_batch_learning_rate=5e-05_
../test_models/ml_models_saved/dl_models/tensor_logs/run_2022_05_10_06_04_54_rezk_skipgram_NS_word2vec_Rmsprob_lstm_with_batch_learning_rate=5e-05_


2022-05-10 06:04:54.934719: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-10 06:04:54.945514: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-10 06:04:54.947337: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-10 06:04:55.407826: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-10 06:04:55.409639: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 132, 300)         1200      
 ormalization)                                                   
                                                                 
 lstm (LSTM)                 (None, 132, 25)           32600     
                                                                 
 batch_normalization_1 (Batc  (None, 132, 25)          100       
 hNormalization)                                                 
                                                                 
 dropout (Dropout)           (None, 132, 25)           0         
                                                                 
 flatten (Flatten)           (None, 3300)              0         
                                                                 
 dense (Dense)               (None, 18)                5

In [9]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

Epoch 1/10


2022-05-10 06:05:07.818662: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


301/301 [==============================] - 15s 31ms/step - loss: 1.3518 - accuracy: 0.5803 - val_loss: 1.4356 - val_accuracy: 0.4873 - lr: 5.0000e-05
Epoch 2/10
301/301 [==============================] - 8s 27ms/step - loss: 1.1128 - accuracy: 0.6233 - val_loss: 1.0729 - val_accuracy: 0.6548 - lr: 5.0000e-05
Epoch 3/10
301/301 [==============================] - 8s 27ms/step - loss: 1.0418 - accuracy: 0.6419 - val_loss: 1.0488 - val_accuracy: 0.6599 - lr: 5.0000e-05
Epoch 4/10
301/301 [==============================] - 8s 25ms/step - loss: 0.9918 - accuracy: 0.6538 - val_loss: 1.0266 - val_accuracy: 0.6396 - lr: 5.0000e-05
Epoch 5/10
301/301 [==============================] - 8s 26ms/step - loss: 0.9560 - accuracy: 0.6565 - val_loss: 1.0170 - val_accuracy: 0.6599 - lr: 5.0000e-05
Epoch 6/10
301/301 [==============================] - 7s 25ms/step - loss: 0.9225 - accuracy: 0.6678 - val_loss: 1.0098 - val_accuracy: 0.6599 - lr: 5.0000e-05
Epoch 7/10
301/301 [==============================

# Load best model & predict test set

In [10]:
strat_test_set = read_file("test/strat_test_set.csv")
strat_test_set.head()

,text,label,classes
0,نرجوا عدم متابعه وحظر حسابات : المباحث تابعني ...,2,OBJ
1,المسلماني اغلي متحدث اعلامي للرئيس واشهر كذاب ...,0,NEG
2,الاحد 3 يوليو : ابو الفتوح في ندوه بالمؤتمر ال...,2,OBJ
3,#كبسوله_صحيه #صحه #طفل #طفلي #نوم ##نوبه #نصيحه,2,OBJ
4,#اللي_رافضين_السيسي_رئيس_بيعملوا_فولوا_لبعض #م...,1,NEU


In [11]:
X_test_text = list(strat_test_set['text'])
y_test = strat_test_set['label'].values

In [12]:
X_test_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(X_test_text)

print("Before Tokenization : \n", X_test_text[:3])
print("="*50)
print("After Tokenization : \n", X_test_text_tokenized[:3])
print("="*50)

fullgram_X_test_text_tokenized = get_all_ngrams(X_test_text_tokenized)
print("full gram tokenization : \n", fullgram_X_test_text_tokenized[:3])
print("="*50)

Before Tokenization : 
 ['نرجوا عدم متابعه وحظر حسابات : المباحث تابعني واتابعك زياده المتابعين الاخبار المحليه #الشعب_يقول_كلمته #جماعه_انصار_بيت_طنيطر #الرياض #جده', 'المسلماني اغلي متحدث اعلامي للرئيس واشهر كذاب ومنافق ومضلل ومحرض تحول الي قليل الادب وضع معارضي الجيش في سله القمامه كلام زباله', 'الاحد 3 يوليو : ابو الفتوح في ندوه بالمؤتمر السنوي لهندسه عين شمس 5 مساء']
After Tokenization : 
 [['نرجوا', 'عدم', 'متابعه', 'وحظر', 'حسابات', ':', 'المباحث', 'تابعني', 'واتابعك', 'زياده', 'المتابعين', 'الاخبار', 'المحليه', '#', 'الشعب_يقول_كلمته', '#', 'جماعه_انصار_بيت_طنيطر', '#', 'الرياض', '#', 'جده'], ['المسلماني', 'اغلي', 'متحدث', 'اعلامي', 'للرئيس', 'واشهر', 'كذاب', 'ومنافق', 'ومضلل', 'ومحرض', 'تحول', 'الي', 'قليل', 'الادب', 'وضع', 'معارضي', 'الجيش', 'في', 'سله', 'القمامه', 'كلام', 'زباله'], ['الاحد', '3', 'يوليو', ':', 'ابو', 'الفتوح', 'في', 'ندوه', 'بالمؤتمر', 'السنوي', 'لهندسه', 'عين', 'شمس', '5', 'مساء']]
full gram tokenization : 
 [['نرجوا', 'عدم', 'متابعه', 'وحظر', 'حسابات', ':'

In [13]:
rezk_model = keras_load_model("../ml_models_saved/dl_models/run_with_rezk_skipgram_NS_word2vec_Rmsprob_lstm_with_batch_learning_rate=5e-05__model.h5"  )

X_test_embed_matrix = text_to_matrix_using_word2vec(our_word2vec_model, fullgram_X_test_text_tokenized, max_len_str)
X_test_embed_matrix = X_test_embed_matrix.reshape(X_test_embed_matrix.shape[0], max_len_str, number_of_features)

keras_f1_score_result(rezk_model, X_test_embed_matrix, y_test)
print("="*50)

(201, 132, 300)
(201, 39600)
[-0.05737  -0.2964    0.4402   -0.03635   0.2576   -0.579     0.0986
  0.9897    0.005226 -0.2986    0.1382   -0.047    -0.355    -0.5176
  0.1824    0.00398  -0.115    -0.398     0.4712   -0.6914    0.0857
  0.3457    0.01616  -0.4534   -0.04105  -0.1515   -0.525     0.3433
 -0.07227  -0.6094    0.663    -0.7783    0.1589    0.2815   -0.2023
  0.04883  -0.2415    0.08405   0.2712    0.0965    0.236    -0.2524
  0.059    -0.3374   -0.3354    0.4438   -0.3762    0.348     0.00602
 -0.4531  ]
===================== Validate Result =====================
F1 score is:  0.6915422885572139
